In [1]:
## Another common problem for NLP Problems is Seq2Seq Problems

In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
 # Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-03 15:34:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-03 15:34:51 (86.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [6]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-09-03 15:34:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.207, 209.85.145.207, 142.250.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-09-03 15:34:56 (117 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##### The dataset we are going to use for deep learning is `Kaggle Distaster Dataset`

In [4]:
## How better to become one with the data?
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [9]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [10]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}", "(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("--\n")

Target:1 (real disaster)
Text:
How long O Lord (Study 3)
 The sixth seal opens the events of Revelation 12. The political upheaval in the Roman... http://t.co/GW0CXoOJyV

--

Target:1 (real disaster)
Text:
Property losses from #California wildfire nearly double as week-old blaze rages: The fireÛ_ http://t.co/MsdizftZ2g

--

Target:1 (real disaster)
Text:
11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...

--

Target:0 (not real disaster)
Text:
Remember how Nora Jones crushed it in Two Weeks Notice?

--

Target:0 (not real disaster)
Text:
Coursing* '@WEYREY_gidi: Now they are causing Di Maria.. LOL'

--



In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                       train_df_shuffled['target'].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [15]:
len(train_df_shuffled)

7613

In [16]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [15]:
text_vectorizer=TextVectorization(max_tokens=None,
                                  standardize="lower_and_strip_punctuation",
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  )

In [19]:
round(sum(len(i.split()) for i in train_sentences)/len(train_sentences))

15

In [20]:
max_vocab_length=10000
max_length=15

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  output_mode='int',
                                  output_sequence_length=max_length)

In [21]:
text_vectorizer.adapt(train_sentences)

# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentence=random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Beautiful disaster // Jon McLaughlin is such a good song

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 588,   75, 3714,    1,    9,  666,    3,  136,  528,    0,    0,
           0,    0,    0,    0]])>

In [23]:
word_in_vocab=text_vectorizer.get_vocabulary()
print(word_in_vocab[:5])
word_in_vocab[-5:]


['', '[UNK]', 'the', 'a', 'in']


['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [24]:
print(len(word_in_vocab))

10000


In [25]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # Set input shape
                           output_dim=128, # Set output shape
                           input_length=max_length) # Set how long each input should be

embedding

In [27]:
random_sentence=random.choice(train_sentences)
print(f"Original text:\n {random_sentence}")
sample_embedding=embedding(text_vectorizer([random_sentence]))
sample_embedding

Original text:
 Messi: has tattoos so he can't donate blood
Ronaldo: runs into burning buildings to save dogs #respect


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03532723,  0.04424863,  0.00896315, ...,  0.03214567,
         -0.00154469,  0.00958459],
        [-0.01606043, -0.00670875, -0.04788568, ..., -0.0218099 ,
          0.04101909,  0.04126951],
        [ 0.03663528, -0.01093787,  0.02369492, ...,  0.01998233,
         -0.03015897, -0.03764578],
        ...,
        [-0.03829162,  0.01588632,  0.04400953, ..., -0.04880197,
          0.01212034,  0.01275796],
        [-0.03769428, -0.00450997,  0.03449098, ...,  0.0235734 ,
          0.02259197, -0.03824301],
        [ 0.04396969, -0.01714847, -0.01344179, ...,  0.03060024,
         -0.03180718, -0.03901184]]], dtype=float32)>

In [28]:
sample_embedding[0][0],sample_embedding[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03532723,  0.04424863,  0.00896315, -0.03461763,  0.03434597,
        -0.00651692, -0.02187118,  0.04320529, -0.02807523, -0.00250275,
        -0.03814435,  0.01108231,  0.00833005, -0.01412891,  0.02483796,
        -0.00806398, -0.00125035, -0.02167287, -0.03684282, -0.00394477,
         0.01184207, -0.01568198, -0.04572482,  0.00488933, -0.02928839,
         0.00925233,  0.03507903,  0.02915097, -0.02952743, -0.04218463,
        -0.02933844,  0.02789335, -0.018034  ,  0.01596259, -0.04863076,
         0.00676094,  0.04092449, -0.02262682,  0.01459619,  0.00726004,
        -0.03702463,  0.04739043, -0.03789669,  0.01675494, -0.03427006,
        -0.00770553,  0.00075557, -0.01396566,  0.03163595, -0.00744382,
         0.01003266, -0.02771249, -0.01081917, -0.02642369, -0.04154788,
        -0.02483526,  0.04520369, -0.04363137, -0.01676164,  0.01164594,
        -0.03356995, -0.03203795,  0.03187878, -0.01553041, -0.04275223,
  

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_0 = Pipeline([
    ('tfidf',TfidfVectorizer()), # Converting text into numbers
    ('clf',MultinomialNB()) # naive bayes used to model text
])

#Fit the pipeline
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
baseline_score=model_0.score(val_sentences,val_labels)

print(f"Our baseline model achieves an accuracy of {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of 79.27%


In [37]:
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [42]:
val_labels[:20]==baseline_preds[:20]

array([False, False,  True, False, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False, False])

In [27]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average='weighted')
  model_results={"accuracy":model_accuracy,
                 'precision':model_precision,
                 'recall':model_recall,
                 'f1':model_f1,
                 }
  return model_results

In [46]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [16]:

from helper_functions import create_tensorboard_callback

In [17]:
SAVE_DIR = "model_logs"

In [116]:
model_2.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15, 1)               │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name='model_1_dense')

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
model_1_history=model_1.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name='simple_dense_model')])

Saving TensorBoard log files to: model_logs/simple_dense_model/20240903-093017
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8134 - loss: 0.4587 - val_accuracy: 0.7782 - val_loss: 0.4744
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8579 - loss: 0.3535 - val_accuracy: 0.7887 - val_loss: 0.4606
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8978 - loss: 0.2766 - val_accuracy: 0.7953 - val_loss: 0.4772
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9123 - loss: 0.2322 - val_accuracy: 0.7861 - val_loss: 0.4855
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9249 - loss: 0.2051 - val_accuracy: 0.7887 - val_loss: 0.5021


In [65]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7780 - loss: 0.5374 


[0.5021362900733948, 0.7887139320373535]

In [66]:
embedding.weights

[<KerasVariable shape=(10000, 128), dtype=float32, path=embedding/embeddings>]

In [76]:
embed_weights=model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
!tensorboard dev upload --logdir ./  \
--name
--description
--one_shot

In [ ]:
!tensorboard dev delete --experiment_id ""

In [110]:
model_1_pred_probs.shape

(762, 1)

In [88]:
model_1_pred_probs=model_1.predict(val_sentences)
print(model_1_pred_probs.ndim)
model_1_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2


array([[0.5124168 ],
       [0.81957936],
       [0.9991414 ],
       [0.09528156],
       [0.0715462 ],
       [0.9610917 ],
       [0.9358293 ],
       [0.9969579 ],
       [0.98623157],
       [0.28367364]], dtype=float32)

In [91]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
print(model_1_preds.ndim)
model_1_preds[:20]

1


<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [92]:
model_1_results=calculate_results(val_labels,
                                  model_1_preds)

In [93]:
model_1_results

{'accuracy': 0.7887139107611548,
 'precision': 0.7906616506949444,
 'recall': 0.7887139107611548,
 'f1': 0.7868569139200134}

In [102]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False,  True])

In [100]:
list(baseline_results.values())

[0.7926509186351706,
 0.8111390004213173,
 0.7926509186351706,
 0.7862189758049549]

In [105]:
# def compare_baseline_to_new_results(baseline_results, new_model_results):
#   for key, value in baseline_results.items():
#     print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

# compare_baseline_to_new_results(baseline_results=baseline_results,
#                                 new_model_results=model_1_results)


def compare_baseline_to_new_results(baseline_result,new_model_results):
  for key,value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]}, Difference: {new_model_results[key]-baseline_results[key]}")

In [106]:
compare_baseline_to_new_results(baseline_results,model_1_results)

Baseline accuracy: 0.79, New accuracy: 0.7887139107611548, Difference: -0.003937007874015741
Baseline precision: 0.81, New precision: 0.7906616506949444, Difference: -0.02047734972637283
Baseline recall: 0.79, New recall: 0.7887139107611548, Difference: -0.003937007874015741
Baseline f1: 0.79, New f1: 0.7868569139200134, Difference: 0.0006379381150585228


In [118]:
words_in_vocab=text_vectorizer.get_vocabulary()

In [119]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [120]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [125]:
embed_weights=model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [129]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [130]:
try:
  from google.colab import files
  files.download("vectors.tsv")
  files.download('metadata.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

```Input->tokenizer->Embedding->LSTM/Dense->Output(label_probability)```

In [136]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
# print(x.shape)
# x=layers.LSTM(64,return_sequences=True)(x)  # When you are setting rnn's cells together you need to set return_sequences=True
# print(x.shape)
x=layers.LSTM(64)(x)
# print(x.shape)
# x=layers.Dense(64,activation='relu')(x)
# print(x.shape)
outputs=layers.Dense(1,activation='sigmoid')(x)
# print(outputs.shape)
model_2=tf.keras.Model(inputs,outputs,name='model_2_LSTM')

In [138]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [139]:
model_2.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [141]:
model_2_history=model_2.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'LSTM')])

Saving TensorBoard log files to: model_logs/LSTM/20240903-115519
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.9287 - loss: 0.2586 - val_accuracy: 0.7835 - val_loss: 0.5768
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9485 - loss: 0.1389 - val_accuracy: 0.7782 - val_loss: 0.6737
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9536 - loss: 0.1210 - val_accuracy: 0.7874 - val_loss: 0.7238
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9649 - loss: 0.0902 - val_accuracy: 0.7848 - val_loss: 0.7988
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9703 - loss: 0.0762 - val_accuracy: 0.7808 - val_loss: 0.8889


In [142]:
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step


array([[0.00749222],
       [0.87450296],
       [0.9997619 ],
       [0.09813866],
       [0.00118282],
       [0.9987272 ],
       [0.9845456 ],
       [0.999834  ],
       [0.9997236 ],
       [0.43534318]], dtype=float32)

In [144]:
model_2_preds=tf.squeeze(tf.round(model_2.predict(val_sentences)))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [145]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [147]:
model_2_results=calculate_results(val_labels,model_2_preds)
model_2_results

{'accuracy': 0.7808398950131233,
 'precision': 0.7834583238190046,
 'recall': 0.7808398950131233,
 'f1': 0.778533312750939}

In [148]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [159]:
np.array(list(baseline_results.values()))==np.array(list(model_2_results.values()))

array([False, False, False, False])

The GRU has similar features as compared to LSTM cells but it has less parameters

In [168]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
# print(x.shape)
x=layers.GRU(64)(x)
# print(x.shape)
# x=layers.LSTM(42,return_sequences=True)(x)
# print(x.shape)
# x=layers.GRU(99)(x)
# print(x.shape)
# x=layers.Dense(64,activation='relu')(x)
# print(x.shape)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs,name='model_3_GRU')
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_11 (GRU)                         │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [170]:
model_3.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])
model_3_history=model_3.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'GRU')])

Saving TensorBoard log files to: model_logs/GRU/20240903-121952
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.8888 - loss: 0.2647 - val_accuracy: 0.7743 - val_loss: 0.8526
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9720 - loss: 0.0767 - val_accuracy: 0.7795 - val_loss: 0.8150
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.9749 - loss: 0.0643 - val_accuracy: 0.7743 - val_loss: 0.9639
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9769 - loss: 0.0577 - val_accuracy: 0.7848 - val_loss: 1.0594
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9796 - loss: 0.0522 - val_accuracy: 0.7769 - val_loss: 1.1410


In [171]:
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


array([[5.04496787e-03],
       [6.56286776e-01],
       [9.99714494e-01],
       [4.11798507e-02],
       [1.17112606e-04],
       [9.99614716e-01],
       [9.65287983e-01],
       [9.99913096e-01],
       [9.99814212e-01],
       [5.48448801e-01]], dtype=float32)

In [173]:
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [174]:
model_3_results=calculate_results(val_labels,model_3_pred)
model_3_results

{'accuracy': 0.7769028871391076,
 'precision': 0.7784265812728361,
 'recall': 0.7769028871391076,
 'f1': 0.7750036673315875}

In [187]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
# print(x.shape)
x=layers.Bidirectional(layers.LSTM(64))(x)
# print(x.shape)
# x=layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
outputs=layers.Dense(1,activation='sigmoid')(x)
# print(outputs.shape)
model_4=tf.keras.Model(inputs,outputs,name='model_4_Bidirectional')


(None, 128)


In [188]:
model_4.summary()

Model: "model_4_Bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_8 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [190]:
model_4.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'bidirectional')])

Saving TensorBoard log files to: model_logs/bidirectional/20240903-125031
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.9517 - loss: 0.1904 - val_accuracy: 0.7769 - val_loss: 0.9768
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.9795 - loss: 0.0507 - val_accuracy: 0.7690 - val_loss: 1.1367
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9777 - loss: 0.0489 - val_accuracy: 0.7677 - val_loss: 1.2670
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.9807 - loss: 0.0428 - val_accuracy: 0.7690 - val_loss: 1.3925
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9814 - loss: 0.0431 - val_accuracy: 0.7664 - val_loss: 1.4029


In [191]:
model_4_pred_prob=model_4.predict(val_sentences)
model_4_pred_prob[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


array([[4.9850306e-01],
       [6.9302589e-01],
       [9.9996191e-01],
       [5.5403687e-02],
       [1.2067959e-05],
       [9.9958342e-01],
       [9.8140842e-01],
       [9.9998260e-01],
       [9.9997175e-01],
       [9.8817098e-01]], dtype=float32)

In [193]:
model_4_pred=tf.squeeze(tf.round(model_4_pred_prob))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [194]:
model_4_results=calculate_results(val_labels,model_4_pred)
model_4_results

{'accuracy': 0.7664041994750657,
 'precision': 0.7667302683900181,
 'recall': 0.7664041994750657,
 'f1': 0.7650123935123555}

# Convolution Neural Network for text(and other types of sequences)
Architecture of Conv1D Sequence Models

In [198]:
from tensorflow.keras import layers
embedding_test=embedding(text_vectorizer(['this is a test sentence']))
conv_1d=layers.Conv1D(filters=64,kernel_size=5,strides=1,activation='relu',padding='same')
conv_1d_output=conv_1d(embedding_test)
max_pool=layers.GlobalMaxPool1D()
max_pool_output=max_pool(conv_1d_output)

embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [202]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,strides=1,activation='relu',padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name='model_5_Conv1D')
model_5.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])


model_5.summary()


Model: "model_5_Conv1D"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)          │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_4               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [204]:
model_5_history=model_5.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=[val_sentences,val_labels],
            callbacks=[create_tensorboard_callback(SAVE_DIR,'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20240903-132348
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9571 - loss: 0.1670 - val_accuracy: 0.7769 - val_loss: 0.8897
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9717 - loss: 0.0724 - val_accuracy: 0.7638 - val_loss: 1.0643
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9796 - loss: 0.0555 - val_accuracy: 0.7677 - val_loss: 1.1231
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9815 - loss: 0.0460 - val_accuracy: 0.7598 - val_loss: 1.1672
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9782 - loss: 0.0503 - val_accuracy: 0.7677 - val_loss: 1.1851


In [205]:
model_5_pred_probs=model_5.predict(val_sentences)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [206]:
model_5_pred_probs[:10]

array([[8.8746034e-02],
       [9.5329744e-01],
       [9.9996018e-01],
       [3.9373029e-02],
       [1.4474533e-07],
       [9.9763268e-01],
       [9.9669558e-01],
       [9.9997926e-01],
       [9.9999976e-01],
       [7.8314561e-01]], dtype=float32)

In [207]:
model_5_preds=tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [210]:
model_5.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7396 - loss: 1.3747


[1.1851104497909546, 0.7677165269851685]

In [211]:
model_5_results=calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 0.7677165354330708,
 'precision': 0.7674148766835972,
 'recall': 0.7677165354330708,
 'f1': 0.7669760007914206}

In [213]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [214]:
np.array(list(model_5_results.values()))==np.array(list(baseline_results.values()))

array([False, False, False, False])

In [215]:
np.array(list(model_5_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

In [216]:
np.array(list(model_5_results.values()))-np.array(list(baseline_results.values()))

array([-0.02493438, -0.04372412, -0.02493438, -0.01924298])

In [2]:
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [49]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation: te

In [3]:
tf.__version__

'2.15.0'

In [220]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [226]:
embed_samples[1].shape

TensorShape([512])

In [18]:
sentence_encoder_layer=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                  input_shape=[],
                                      dtype=tf.string,
                                      name='USE',
                                      trainable=False)

In [19]:
import tf_keras

In [20]:
from tf_keras import Sequential
from tf_keras import layers

In [21]:
model_6 = tf_keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])


In [22]:
from helper_functions import create_tensorboard_callback

In [23]:
model_6_history=model_6.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'USE')])


Saving TensorBoard log files to: model_logs/USE/20240903-154316


Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/5
215/215 [==============================] - 34s 25ms/step - loss: 0.5046 - accuracy: 0.7758 - val_loss: 0.4555 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4175 - accuracy: 0.8162 - val_loss: 0.4461 - val_accuracy: 0.8123
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4028 - accuracy: 0.8206 - val_loss: 0.4432 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3956 - accuracy: 0.8244 - val_loss: 0.4363 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3884 - accuracy: 0.8282 - val_loss: 0.4319 - val_accuracy: 0.8150


In [24]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 12ms/step


array([[0.16955325],
       [0.78207767],
       [0.9805038 ],
       [0.17457776],
       [0.7213493 ],
       [0.72538084],
       [0.9746585 ],
       [0.97061175],
       [0.9235164 ],
       [0.0804296 ]], dtype=float32)

In [25]:
model_6_pred=tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [28]:
model_6_results=calculate_results(val_labels,model_6_pred)
model_6_results

{'accuracy': 0.8149606299212598,
 'precision': 0.8166816944448522,
 'recall': 0.8149606299212598,
 'f1': 0.8136303789632117}

In [29]:
model_6.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 11ms/step - loss: 0.4319 - accuracy: 0.8150


[0.4319266676902771, 0.8149606585502625]

In [30]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrap  (None, 512)               256797824 
 per)                                                            
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [66]:
len(train_df)

7613

In [68]:
train_Df

6851

In [70]:
train_10_percent=train_df_shuffled.sample(frac=0.1,random_state=42)
len(train_10_percent)

761

In [83]:
train_sentences_10_percent=train_10_percent['text'].to_list()
train_labels_10_percent=train_10_percent['target'].to_list()

In [73]:
train_labels_10_percent[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [75]:
len(train_sentences_10_percent),len(train_labels_10_percent)

(761, 761)

In [76]:
train_10_percent.target.value_counts()

,count
target,
0,413
1,348


In [80]:
train_df_shuffled.target.value_counts()

,count
target,
0,4342
1,3271


In [81]:
model_7=tf_keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid',name='output_layer')
])

model_7.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [84]:
model_7_history=model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'10_percent_USE')])

Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Saving TensorBoard log files to: model_logs/10_percent_USE/20240903-142845
Epoch 1/5
24/24 [==============================] - 4s 52ms/step - loss: 0.6713 - accuracy: 0.6767 - val_loss: 0.6346 - val_accuracy: 0.7874
Epoch 2/5
24/24 [==============================] - 1s 22ms/step - loss: 0.6046 - accuracy: 0.7937 - val_loss: 0.5590 - val_accuracy: 0.7979
Epoch 3/5
24/24 [==============================] - 1s 22ms/step - loss: 0.5293 - accuracy: 0.7884 - val_loss: 0.4922 - val_accuracy: 0.7979
Epoch 4/5
24/24 [==============================] - 1s 26ms/step - loss: 0.4724 - accuracy: 0.8003 - val_loss: 0.4444 - val_accuracy: 0.8176
Epoch 5/5
24/24 [==============================] - 1s 22ms/step - loss: 0.4339 - accuracy: 0.8160 - val_loss: 0.4132 - val_accuracy: 0.8228


In [85]:
model_7_preds_probs=model_7.predict(val_sentences)

24/24 [==============================] - 1s 16ms/step


In [86]:
model_7_preds_probs[:10]

array([[0.24654257],
       [0.80588895],
       [0.89751506],
       [0.33401963],
       [0.7895404 ],
       [0.8410768 ],
       [0.8974214 ],
       [0.9346192 ],
       [0.8345892 ],
       [0.06969212]], dtype=float32)

In [87]:
model_7_preds=tf.squeeze(tf.round(model_7_preds_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [88]:
model_7_results=calculate_results(val_labels,model_7_preds)
model_7_results

{'accuracy': 0.8228346456692913,
 'precision': 0.8226500636770807,
 'recall': 0.8228346456692913,
 'f1': 0.8226266758210372}

In [89]:
model_7.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 10ms/step - loss: 0.4132 - accuracy: 0.8228


[0.4131600558757782, 0.8228346705436707]

In [90]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper_7 (ModuleWr  (None, 512)               256797824 
 apper)                                                          
                                                                 
 dense_12 (Dense)            (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


## Making a better data split so no data leakage

In [31]:
train_10_percent_split=int(0.1*len(train_sentences))
train_sentences_10_percent=train_sentences[:train_10_percent_split]
train_labels_10_percent=train_labels[:train_10_percent_split]
len(train_sentences_10_percent),len(train_labels_10_percent)

(685, 685)

In [32]:
train_sentences_10_percent[:10],train_labels_10_percent[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [33]:
pd.Series(np.array(train_labels_10_percent)).value_counts()

,count
0,406
1,279


In [34]:
model_7=tf_keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid',name='output_layer')
])

model_7.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [35]:
model_7_history=model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'10_percent_USE')])

Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Saving TensorBoard log files to: model_logs/10_percent_USE/20240903-154457
Epoch 1/5
22/22 [==============================] - 4s 64ms/step - loss: 0.6623 - accuracy: 0.7080 - val_loss: 0.6433 - val_accuracy: 0.7100
Epoch 2/5
22/22 [==============================] - 1s 26ms/step - loss: 0.5922 - accuracy: 0.7985 - val_loss: 0.5855 - val_accuracy: 0.7638
Epoch 3/5
22/22 [==============================] - 1s 44ms/step - loss: 0.5177 - accuracy: 0.8190 - val_loss: 0.5370 - val_accuracy: 0.7677
Epoch 4/5
22/22 [==============================] - 1s 45ms/step - loss: 0.4598 - accuracy: 0.8248 - val_loss: 0.5027 - val_accuracy: 0.7769
Epoch 5/5
22/22 [==============================] - 1s 45ms/step - loss: 0.4191 - accuracy: 0.8394 - val_loss: 0.4916 - val_accuracy: 0.7822


In [36]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper_1 (ModuleWr  (None, 512)               256797824 
 apper)                                                          
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [37]:
model_7.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 15ms/step - loss: 0.4916 - accuracy: 0.7822


[0.4916248023509979, 0.7821522355079651]

In [38]:
model_7_preds_probs=model_7.predict(val_sentences)
model_7_preds=tf.squeeze(tf.round(model_7_preds_probs))
model_7_results=calculate_results(val_labels,model_7_preds)
model_7_results


print(model_7_preds_probs[:10])

24/24 [==============================] - 1s 11ms/step
[[0.19161098]
 [0.55152506]
 [0.89958495]
 [0.35104242]
 [0.50233936]
 [0.676862  ]
 [0.8676804 ]
 [0.8052875 ]
 [0.82572025]
 [0.14269684]]


In [39]:
print(model_7_preds[:10])

tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)


In [ ]:
model_7_results

In [29]:
all_model_results=pd.DataFrame({"model_6_results":model_6_results,
                                'model_7_results':model_7_results})
all_model_results

NameError: name 'model_6_results' is not defined

In [ ]:
all_model_results=all_model_results.transpose()

In [ ]:
all_model_results

In [30]:
all_model_results.plot(kind='bar',figsize=(10,7)).legend(bbox_to_anchor=(1,1))

NameError: name 'all_model_results' is not defined

In [31]:
all_model_results.sort_values('f1',ascending=False)['f1'].plot(kind='bar',figsize=(10,7))

NameError: name 'all_model_results' is not defined

In [ ]:
!tensorboard dev upload --logdir ./model/logs \
--name 'hey' \
--description 'woah'\
--one_shot

In [ ]:
!tensorboard dev list

In [ ]:
!tensorboard dev delete --experiment_id=""

#There is two ways to save your model
1. HDF5 Format(Hierarchical Data Format)
2. SavedModel Format

In [139]:
model_6_results

{'accuracy': 0.8110236220472441,
 'precision': 0.8113631917551808,
 'recall': 0.8110236220472441,
 'f1': 0.8102336058004983}

NotImplementedError: 
Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config

In [144]:
import tensorflow_hub as hub
loaded_model_6=tf.keras.models.load_model("model_6.h5",custom_objects={"KerasLayer":hub.KerasLayer})

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x784560503370>.

In [32]:
model_6.save("model_6_SavedModel_format")

In [33]:
loaded_model_6_SavedModel_format=tf.keras.models.load_model("model_6_SavedModel_format")

ValueError: File format not supported: filepath=model_6_SavedModel_format. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(model_6_SavedModel_format, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
loaded_model_6_SavedModel_format.evaluate(val_sentences,val_labels)

In [ ]:
model_6_results

In [44]:
import pandas as pd
val_df=pd.DataFrame({'text':val_sentences,
                     'labels':val_labels,
                     'pred':model_6_pred,
                     'pred_probs':tf.squeeze(model_6_pred_probs)})

In [45]:
val_df.head()

,text,labels,pred,pred_probs
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.169553
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.782078
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.980504
3,@camilacabello97 Internally and externally scr...,1,0.0,0.174578
4,Radiation emergency #preparedness starts with ...,1,1.0,0.721349


In [53]:
most_wrong=val_df[val_df['pred']!=val_df['labels']].sort_values('pred_probs',ascending=False)

In [54]:
most_wrong[:10]

,text,labels,pred,pred_probs
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.893221
759,FedEx will no longer transport bioterror patho...,0,1.0,0.874861
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.835293
49,@madonnamking RSPCA site multiple 7 story high...,0,1.0,0.825950
628,@noah_anyname That's where the concentration c...,0,1.0,0.816455
393,@SonofLiberty357 all illuminated by the bright...,0,1.0,0.815730
251,@AshGhebranious civil rights continued in the ...,0,1.0,0.795420
109,[55436] 1950 LIONEL TRAINS SMOKE LOCOMOTIVES W...,0,1.0,0.794066
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.782078
698,åÈMGN-AFRICAå¨ pin:263789F4 åÈ Correction: Ten...,0,1.0,0.774874


In [55]:
for row in most_wrong[:10].itertuples():
  _,text,target,pred,prob=row
  print(f"Target: {target}, Pred: {int(pred)},Prob is {prob}")
  print(f"Text is {text}")
  print("--\n\n")

Target: 0, Pred: 1,Prob is 0.8932211995124817
Text is ? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying
--


Target: 0, Pred: 1,Prob is 0.874861478805542
Text is FedEx will no longer transport bioterror pathogens in wake of anthrax lab mishaps http://t.co/lHpgxc4b8J
--


Target: 0, Pred: 1,Prob is 0.8352929949760437
Text is Ashes 2015: AustraliaÛªs collapse at Trent Bridge among worst in history: England bundled out Australia for 60 ... http://t.co/t5TrhjUAU0
--


Target: 0, Pred: 1,Prob is 0.8259497284889221
Text is @madonnamking RSPCA site multiple 7 story high rise buildings next to low density character residential in an area that floods
--


Target: 0, Pred: 1,Prob is 0.8164553046226501
Text is @noah_anyname That's where the concentration camps and mass murder come in. 
 
EVERY. FUCKING. TIME.
--


Target: 0, Pred: 1,Prob is 0.8157302737236023
Text is @SonofLiberty357 all illuminated by the brightly burning buildings all around the town!
--


Target: 0, Pred: 

In [56]:
for row in most_wrong[-10:].itertuples():
  _, text, target, pred, prob = row
  print(f"Target: {target}, Pred: {int(pred)}, Prob: {prob}")
  print(f"Text:\n{text}\n")
  print("----\n")

Target: 1, Pred: 0, Prob: 0.05390774831175804
Text:
@DavidVonderhaar At least you were sincere ??

----

Target: 1, Pred: 0, Prob: 0.05385352298617363
Text:
Lucas Duda is Ghost Rider. Not the Nic Cage version but an actual 'engulfed in flames' badass. #Mets

----

Target: 1, Pred: 0, Prob: 0.05203240364789963
Text:
Perspectives on the Grateful Dead: Critical Writings (Contributions to the Study http://t.co/fmu0fnuMxf http://t.co/AgGRyhVXKr

----

Target: 1, Pred: 0, Prob: 0.04929757118225098
Text:
@willienelson We need help! Horses will die!Please RT &amp; sign petition!Take a stand &amp; be a voice for them! #gilbert23 https://t.co/e8dl1lNCVu

----

Target: 1, Pred: 0, Prob: 0.047503482550382614
Text:
You can never escape me. Bullets don't harm me. Nothing harms me. But I know pain. I know pain. Sometimes I share it. With someone like you.

----

Target: 1, Pred: 0, Prob: 0.035701118409633636
Text:
I get to smoke my shit in peace

----

Target: 1, Pred: 0, Prob: 0.03421356528997421
Te

In [57]:
# Making predictions on the test dataset
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6.predict([test_sample])) # has to be list
  pred = tf.round(pred_prob)
  print(f"Pred: {int(pred)}, Prob: {pred_prob}")
  print(f"Text:\n{test_sample}\n")
  print("----\n")

1/1 [==============================] - 0s 60ms/step
Pred: 0, Prob: 0.17365755140781403
Text:
AND I'M DROWNING IN THE DÌäJÌÛ VUUUUU WE'VE SEEN IT ALL BEFOOOOOOOORE

----

1/1 [==============================] - 0s 44ms/step
Pred: 1, Prob: 0.9643959999084473
Text:
Bombing of Hiroshima 1945. http://t.co/4UHq9jKCvq

----

1/1 [==============================] - 0s 45ms/step
Pred: 1, Prob: 0.7461358308792114
Text:
#Irvine #SigAlert update: 133 S now closed at Irvine Blvd. 133 N closed past the 5 b/c of trash truck fire. @KNX1070

----

1/1 [==============================] - 0s 49ms/step
Pred: 1, Prob: 0.9837262630462646
Text:
#?? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ...  http://t.co/ykLruIMIsD

----

1/1 [==============================] - 0s 58ms/step
Pred: 0, Prob: 0.024897390976548195
Text:
So if you secretly have a crush on me and can sing lmk lmfao

----

1/1 [==============================] - 0s 56ms/step
Pred: 1, Prob: 0.949187040328

### The speed/score tradeoff

In [60]:
import time

def pred_timer(model,samples):
  """ Time how long a model takes to make predictions on input samples """
  start_time=time.perf_counter()
  model.predict(samples)
  end_time=time.perf_counter()
  total_time=end_time-start_time

  print(f"Total prediction time: {round(total_time,2)}s")
  print(f'Average prediction time is {total_time/len(samples)}')


In [61]:
pred_timer(model_6,val_sentences) # For Model 6

24/24 [==============================] - 0s 13ms/step
Total prediction time: 0.73s
Average prediction time is 0.0009558373674543613


In [63]:
model_6_calculate_results=calculate_results(val_labels,model_6_pred)

In [64]:
model_6_calculate_results

{'accuracy': 0.8149606299212598,
 'precision': 0.8166816944448522,
 'recall': 0.8149606299212598,
 'f1': 0.8136303789632117}

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))
plt.scatter(baseline_time_per_pred, baseline_results["f1"], label="baseline")
plt.scatter(model_6_time_per_pred, model_6_results["f1"], label="tf_hub_sentence_encoder")
plt.legend()
plt.title("F1-score versus time per prediction")
plt.xlabel("Time per prediction")
plt.ylabel("F1-Score");